# FIXED COUPON BOND EXAMPLE APPLE CORP

This is a  bond analysis based on example in https://data.bloomberglp.com/bat/sites/3/2017/07/SF-2017_Paul-Fjeldsted.pdf

In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [3]:
from financepy.finutils import *
from financepy.products.bonds import *

###################################################################
# FINANCEPY BETA Version 0.180 - This build: 22 Sep 2020 at 22:13 #
#     This software is distributed FREE & WITHOUT ANY WARRANTY    #
# For info and disclaimer - https://github.com/domokane/FinancePy #
###################################################################



# Define the Bond

In [4]:
issueDate = FinDate(13, 5, 2010)
maturityDate = FinDate(13, 5, 2022)
coupon = 0.027
freqType = FinFrequencyTypes.SEMI_ANNUAL
accrualType = FinDayCountTypes.THIRTY_E_360
face = ONE_MILLION

In [5]:
bond = FinBond(issueDate, maturityDate, coupon, freqType, accrualType, face)

In [6]:
cleanPrice = 101.581564  # if face is 1 then this must be 0.99780842

You can get information about the bond using the print method.

In [7]:
print(bond)

OBJECT TYPE: FinBond
ISSUE DATE: THU 13 MAY 2010
MATURITY DATE: FRI 13 MAY 2022
COUPON: 0.027
FREQUENCY: FinFrequencyTypes.SEMI_ANNUAL
ACCRUAL TYPE: FinDayCountTypes.THIRTY_E_360
FACE AMOUNT: 1000000


## Bond Cash Flows

We first need to set the settlement date of the bond. 

In [8]:
settlementDate = FinDate(21, 7, 2017)

In [9]:
bond.printFlows(settlementDate)

MON 13 NOV 2017      13500.00 
SUN 13 MAY 2018      13500.00 
TUE 13 NOV 2018      13500.00 
MON 13 MAY 2019      13500.00 
WED 13 NOV 2019      13500.00 
WED 13 MAY 2020      13500.00 
FRI 13 NOV 2020      13500.00 
THU 13 MAY 2021      13500.00 
SAT 13 NOV 2021      13500.00 
FRI 13 MAY 2022    1013500.00 


The convention is to use these dates for yield calculations even if some fall on weekends.

## Bond Yield Measures

Current yield is the coupon over the price

In [10]:
bond.currentYield(cleanPrice)*100.0

2.657962620067555

Yield to maturity using UK convention

In [10]:
bond.yieldToMaturity(settlementDate, cleanPrice, FinYTMCalcType.UK_DMO)*100

2.3499999793944197

Yield to maturity using US Street convention

In [11]:
bond.yieldToMaturity(settlementDate, cleanPrice, FinYTMCalcType.US_STREET)*100

2.3499999793944197

Yield to maturity using US Treasury convention

In [12]:
bond.yieldToMaturity(settlementDate, cleanPrice, FinYTMCalcType.US_TREASURY)*100

2.3496418128945775

## Accrued Interest

For consistency let's fix the yield calculation convention to be US Street

In [13]:
yieldConvention = FinYTMCalcType.US_STREET

In [14]:
ytm = bond.yieldToMaturity(settlementDate, cleanPrice, yieldConvention)

Full price is the clean price plus accrued interest

In [15]:
print("Full Price = = %8.6f" %  bond.fullPriceFromYTM(settlementDate, ytm, yieldConvention))

Full Price = = 102.091564


In [16]:
print("Clean Price = %8.6f" % bond.cleanPriceFromYTM(settlementDate, ytm, yieldConvention))

Clean Price = 101.581564


Accrued interest is accrued from previous coupon date to settlement date

In [17]:
print("Previous coupon date is ", bond._pcd)

Previous coupon date is  SAT 13 MAY 2017


In [18]:
print("Settlement date is ", settlementDate)

Settlement date is  FRI 21 JUL 2017


The amount of accrued interest is 

In [19]:
print("Accrued = %12.2f"% bond._accruedInterest)

Accrued =      5100.00


This is based on the following number of days of accrual (for some conventions this is not the actual number of days between last coupon and settlement)

In [20]:
print("Accrued Days = ", bond._accruedDays)

Accrued Days =  68


In [21]:
print("Principal = %12.2f" % bond.principal(settlementDate, ytm, yieldConvention))

Principal =   1015815.64


## Bond Risk Measures

The bond dollar duration is the actual derivative with respect to the yield. It is the bond price change for a 1bp drop in the yield-to-maturity divided by 1bp.

In [22]:
duration = bond.dollarDuration(settlementDate, ytm, yieldConvention)
print("Dollar Duration = ", duration)

Dollar Duration =  456.56933693592805


Modified Duration divides the dollar duration by the full price of the bond

In [23]:
modifiedDuration = bond.modifiedDuration(settlementDate, ytm, yieldConvention)
print("Modified Duration = ", modifiedDuration)

Modified Duration =  4.4721553774602825


Macauley Duration multiplies the dollar duration by (1+y/f) and divides by the full price

In [24]:
macauleyDuration = bond.macauleyDuration(settlementDate, ytm, yieldConvention)
print("Macauley Duration = ", macauleyDuration)

Macauley Duration =  4.524703202684684


Convexity is the second derivative of the bond price with respect to the yield-to-maturity

In [25]:
conv = bond.convexityFromYTM(settlementDate, ytm, yieldConvention)
print("Convexity = ", conv)

Convexity =  0.23024090816492734


Copyright (c) 2020 Dominic O'Kane